In [11]:
import numpy as np
from scipy.special import genlaguerre
from scipy import integrate
import math
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from ipywidgets import interact
from IPython.display import display

## The Wigner function

In [73]:
def wigner(x, p, d = 0, F = np.identity(2)):
    X, P = np.meshgrid(x, p)
    n = 1
    
    r_ave = F @ np.array([0,0]) + d
    cov = F @ (0.5 * np.array([[1,0],[0,1]])) @ F.T
    
    det_cov = np.linalg.det(cov)
    inv_cov = np.linalg.inv(cov)

    W = np.zeros_like(X)
    
    for i in range(X.shape[0]):
        for j in range(X.shape[1]):
            r = np.array([X[i,j],P[i,j]])
            diff = r - r_ave
            W[i,j] = 1/((2*np.pi)**(n) * np.sqrt(det_cov)) * np.exp(-0.5 * diff.T @ inv_cov @ diff)
    return W

## The density matrix (from the Wigner function)

In [86]:
def wigner_nm(x, p, n, m):    
    if m<n:
        poly = genlaguerre(m, n-m)
        return (1/np.pi * np.exp(-X**2 - P**2) *
                (-1)**n *
                (X - 1j*P)**(m-n) *
                np.sqrt(2**(m-n) * math.factorial(n) / math.factorial(m)) *
                poly(2*X**2 + 2*P**2))
    else:
        poly = genlaguerre(n, m-n)
        return np.conjugate(1/np.pi * np.exp(-X**2 - P**2) *
                (-1)**n *
                (X - 1j*P)**(m-n) *
                np.sqrt(2**(m-n) * math.factorial(n) / math.factorial(m)) *
                poly(2*X**2 + 2*P**2))

In [99]:
def density_matrix(x, p, d = 0, F = np.identity(2)):
    rho = np.zeros(4, dtype=np.complex128).reshape(2,2)

    x_incr = x[1] - x[0]
    p_incr = x[1] - x[0]

    X, P = np.meshgrid(x, p)
    
    W = wigner(x, p, d, F)
    
    
    for i in range(2):
        for j in range(2):
            W_nm = wigner_nm(X, P, n = i, m = j)
            rho[i, j] = np.round(np.sum(W * W_nm) * x_incr * p_incr * 2 * np.pi,4)
    return rho

In [101]:
@interact(alpha_real=(-5, 5, 0.1), alpha_imag=(-5, 5, 0.1), s=(-np.pi, np.pi, np.pi/128), theta=(-np.pi, np.pi, np.pi/128), show_signal=True)

def plot_modsig(alpha_real = 1, alpha_imag = 1, s = 0, theta = 0, show_signal=True):
    p = np.linspace(-4, 4, 100)
    x = np.linspace(-4, 4, 100)

    #Displacement
    d = np.array([alpha_real, alpha_imag])
    
    #Squeezing
    F_s = np.cosh(s) * np.identity(2) - np.sinh(s) * np.array([[np.cos(theta),np.sin(theta)],[np.sin(theta),-np.cos(theta)]])
    
    print(density_matrix(x, p, d = d, F = F_s))
    W = wigner(X, P, d = d, F = F_s)
    
    fig = plt.figure(figsize=(8, 7))
    ax = fig.add_subplot(111, projection='3d')
    ax.plot_surface(X, P, W, cmap='viridis')

    ax.set_xlabel('x', fontsize=18)
    ax.set_ylabel('p', fontsize=18)
    ax.text2D(0.75, 0.8, "Wigner function", transform=ax.transAxes, fontsize=18, rotation = -22.5)
    
    ax.view_init(elev=30, azim=135)

    plt.show()

interactive(children=(FloatSlider(value=1.0, description='alpha_real', max=5.0, min=-5.0), FloatSlider(value=1…